In [14]:

# Ensure repository root (with 'Source/ai') is on sys.path
from pathlib import Path
import sys

project_root = next((p for p in [Path.cwd(), *Path.cwd().parents] if (p / 'Source' / 'ai').exists()), None)
if project_root and str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from dotenv import load_dotenv
from langchain_ollama import ChatOllama
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from langchain.tools import Tool
from langgraph.graph import END, StateGraph
import operator
import requests
from Source.ai.Multi_Agent.Source.Main.Tools import poem_tools, sentiment_tools, travel_tools, weather_tools
from Source.ai.Multi_Agent.Source.Main.Agents.Agents_1 import Coordinator_Agent_1, Flight_Agent_1, Hotel_Agent_1, Travel_Agent_1
from Source.ai.Multi_Agent.Source.Main.Agents.Agents_2 import Coordinator_Agent_2, Flight_Agent_2, Hotel_Agent_2, Travel_Agent_2
from Source.ai.Multi_Agent.Source.Main.Agents.Agents_3 import Coordinator_Agent_3, Flight_Agent_3, Hotel_Agent_3, Travel_Agent_3, Aggregator_Agent_3
from Source.ai.Multi_Agent.Source.Main.Agents.Agents_Summary import Abstracter_Agent, Aggregator_Agent, Coordinator_Agent, Evaluator_Agent, Extractor_Agent, GradeCalibrator_Agent, OCR_Agent, SpellChecker_Agent
from typing import TypedDict, Annotated, List, Any, Dict, Literal
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from Source.ai.Multi_Agent.Source.Main.Memory.memory.memory import memory_manager
from Source.ai.Multi_Agent.Source.Main.Memory.memory.long_term_memory import long_term_memory
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import uuid
import os
#from langchain_openai import ChatOpenAI

In [15]:
load_dotenv()

llm = ChatOllama(model="llama3:8b") # <-- Sử dụng model bạn đã kéo về, ví dụ "llama3", "mistral"

class AgentState(TypedDict):
    input: str
    messages: Annotated[List[str], operator.add]
    
prompt = hub.pull("hwchase17/react")

In [4]:
def new_session(user_id: str = "default_user", clear_history: bool = True, keep_preferences: bool = True, auto_continue: bool = False, replay_last_n: int = 20) -> str:
    sid = memory_manager.start_new_session(user_id=user_id, clear_history=clear_history, keep_preferences=keep_preferences)
    print(f"New session started: {sid}")
    if auto_continue:
        initial_state = build_state_from_memory(user_id=user_id, max_messages=replay_last_n)
        run_langgraph_chat(initial_state=initial_state)
    return sid

sid = new_session()

New session started: session_20251028_105421


In [5]:
all_items = long_term_memory.collection.get(include=["metadatas"])
session_ids = sorted({m.get("session_id") for m in all_items["metadatas"] if m})
print(session_ids)

['q', 'session_20250925_130323', 'session_20250925_144950', 'session_20250925_150303', 'session_20250925_151001', 'session_20250927_162902', 'session_20250927_164507', 'session_20250928_092221', 'session_20250928_154914', 'session_20250928_155638', 'session_20251001_160456', 'session_20251002_093329', 'session_20251002_110113', 'session_20251002_143156']


In [16]:
def create_initial_state() -> AgentState:
    return {
        "messages": [],
        "current_agent": "coordinator",
        "needs_user_input": False,
        "conversation_stage": "greeting",
    }
state = create_initial_state()
try:
    state = app.invoke(state, config={"recursion_limit": 20})
except Exception:
    pass

print("Nodes wired: aggregator added between specialist agents and coordinator.")


Nodes wired: aggregator added between specialist agents and coordinator.


In [7]:
def read_long_term_memory_by_session_id(session_id: str):
    col = long_term_memory.collection
    all_items = col.get(include=["documents","metadatas"])
    for doc, meta in zip(all_items["documents"], all_items["metadatas"]):
        if meta.get("session_id") == session_id:
            print(meta.get("timestamp"), meta.get("session_id"), meta.get("role"), ":", doc)

In [9]:
# Khai báo State cho LangGraph
class AgentState(TypedDict):
    messages: List[Any]
    current_agent: str
    needs_user_input: bool
    conversation_stage: Literal["greeting", "reader_ocr", "spellchecker", "extractor", "abstracter", "grade_calibrator", "evaluator", "aggregator", "completed"]
    check: Literal["exactly", "not exactly"] = Field(
        description="Decide if the text is exactly the same as the original text.",
    )
    feedback: str = Field(
        description="If the text is not exactly the same as the original text, provide feedback on how to improve it.",
    )

# Hàm điều hướng sau node coordinator
def decide_next_agent(state: AgentState):
    if state.get("needs_user_input", False):
        return "END"
    return state.get("current_agent", "coordinator_agent")

def route_check(state: AgentState):
    if state["check"] == "exactly":
        return "Accepted"
    elif state["check"] == "not exactly":
        return "Rejected + Feedback"
    
# Build đồ thị LangGraph
workflow = StateGraph(AgentState)

workflow.add_node("coordinator_agent", Coordinator_Agent.coordinator_agent)
workflow.add_node("reader_ocr_agent", OCR_Agent.ocr_agent)
workflow.add_node("spellchecker_agent", SpellChecker_Agent.spellchecker_agent)
workflow.add_node("extractor_agent", Extractor_Agent.extractor_agent)
workflow.add_node("abstracter_agent", Abstracter_Agent.abstracter_agent)
workflow.add_node("grade_calibrator_agent", GradeCalibrator_Agent.grade_calibrator_agent)
workflow.add_node("evaluator_agent", Evaluator_Agent.evaluator_agent)
workflow.add_node("aggregator_agent", Aggregator_Agent.aggregator_agent)

workflow.set_entry_point("coordinator_agent")

workflow.add_conditional_edges(
    "coordinator_agent",
    decide_next_agent,
    {
        "reader_ocr_agent": "reader_ocr_agent",
        "spellchecker_agent": "spellchecker_agent",
        "extractor_agent": "extractor_agent",
        "abstracter_agent": "abstracter_agent",
        "grade_calibrator_agent": "grade_calibrator_agent",
        "evaluator_agent": "evaluator_agent",
        "aggregator_agent": "aggregator_agent",
        "coordinator_agent": "coordinator_agent",
        "END": END,
    },
)
workflow.add_edge("reader_ocr_agent", "spellchecker_agent")
workflow.add_conditional_edges(
    "spellchecker_agent",
    route_check,
    {  
        "Accepted": END,
        "Rejected + Feedback": "reader_ocr_agent",
    },
)
workflow.add_edge("spellchecker_agent", "extractor_agent")
workflow.add_edge("spellchecker_agent", "abstracter_agent")
workflow.add_edge("extractor_agent", "grade_calibrator_agent")
workflow.add_edge("abstracter_agent", "grade_calibrator_agent")
workflow.add_edge("grade_calibrator_agent", "evaluator_agent")
workflow.add_edge("evaluator_agent", "aggregator_agent")
workflow.add_edge("aggregator_agent", END)

app = workflow.compile()

def run_langgraph_chat(initial_state=None):
    print("🤖 Multi-Agent System Summary For Primary School Students")
    print("=" * 60)
    print("Commands: 'exit', 'clear' (STM), 'clear_all' (STM+LTM), 'mem_stats'")

    state = initial_state or create_initial_state()

    # KHÔNG auto-invoke nếu đã có messages (tránh chào lại)
    if not state.get("messages"):
        try:
            state = app.invoke(state, config={"recursion_limit": 50})
            last = state["messages"][-1] if state["messages"] else None
            if last and isinstance(last, AIMessage):
                print(f"\n🤖{state['current_agent']}: {last.content}")
        except Exception:
            pass

    while True:
        if not state.get("needs_user_input", True):
            state = app.invoke(state, config={"recursion_limit": 50})
            last = state["messages"][-1] if state["messages"] else None
            if last and isinstance(last, AIMessage):
                print(f"\n🤖{state['current_agent']}: {last.content}")
            mem = memory_manager.get_memory()
            print(f"   [Memory: {len(mem.conversation_history)} msgs, {len(mem.user_preferences)} prefs]")
            continue

        user_input = input("\n👤 Bạn: ").strip()
        memory_manager.add_message("user", user_input)

        if user_input.lower() in ["exit", "quit", "thoát"]:
            print("👋 Bye MAS Lịch sử chat đã được lưu.")
            break
        if user_input.lower() in ["clear", "xóa", "reset"]:
            memory_manager.clear_memory()
            state = create_initial_state()
            print("🧹 Đã xóa short-term memory. Long-term vẫn giữ.")
            continue
        if user_input.lower() in ["clear_all", "xóa_all", "reset_all"]:
            memory_manager.clear_memory(also_long_term=True)
            state = create_initial_state()
            print("🧹 Đã xóa cả short-term và long-term memory.")
            continue
        if user_input.lower() in ["mem_stats", "memory_stats"]:
            print(f"📊 Long-term Memory: {long_term_memory.collection.count()} items")
            continue

        state["messages"].append(HumanMessage(content=user_input))
        print(f"👤: {user_input}")
        state["needs_user_input"] = False
def build_state_from_memory(user_id: str = "default_user", max_messages: int = 10):
    mem = memory_manager.get_memory(user_id)
    msgs = []
    ctrl = {"thoát","exit","quit","xóa","clear","reset","clear_all","xóa_all","reset_all"}
    for m in mem.conversation_history[-max_messages:]:
        content = (m.get("content") or "").strip()
        if content.lower() in ctrl:
            continue
        role = (m.get("role") or "").lower()
        if role == "user":
            msgs.append(HumanMessage(content=content))
        else:
            msgs.append(AIMessage(content=content))
    needs_user_input = True if msgs and isinstance(msgs[-1], AIMessage) else False
    return {
        "messages": msgs,
        "current_agent": "coordinator_agent",
        "needs_user_input": needs_user_input,
        "conversation_stage": "reader_ocr",
    }
def continue_chat_from_session(session_id: str, user_id: str = "default_user", replay_last_n: int = 20):
    print("Previous chat history:")
    read_long_term_memory_by_session_id("session_20251028_105421")
    loaded = memory_manager.resume_session(session_id, user_id=user_id, replay_last_n=replay_last_n)
    print(f"Resumed {loaded} messages from long-term: {session_id}")
    initial_state = build_state_from_memory(user_id=user_id, max_messages=replay_last_n)
    run_langgraph_chat(initial_state=initial_state)

# run_langgraph_chat()
# hoặc tiếp nối từ 1 session cụ thể:
continue_chat_from_session("session_20251028_105421")

Previous chat history:
Resumed 0 messages from long-term: session_20251028_105421
🤖 Multi-Agent System Summary For Primary School Students
Commands: 'exit', 'clear' (STM), 'clear_all' (STM+LTM), 'mem_stats'

🤖reader_ocr_agent: Xin chào! Tôi là trợ lý xưởng tóm tắt thông minh theo khối lớp. Tôi có thể giúp gì cho bạn?
👤: Tôi muốn tóm tắt văn bản tiếng Việt cho lớp 3

🤖reader_ocr_agent: Rõ ràng! Bạn muốn tóm tắt văn bản tiếng Việt cho lớp 3, đúng không?

Vui mừng! Tôi sẽ giúp bạn thực hiện điều đó.

Văn bản của bạn có thể được đưa lên hình ảnh/PDF hay là text? (You can upload the file/image or type in the text)
   [Memory: 4 msgs, 0 prefs]
👤: Văn bản của tôi như sau: Ngày khai trường đã đến. Sáng sớm, mẹ mới gọi một câu mà tôi đã vùng dậy, khác hẳn mọi ngày. Loáng một cái, tôi đã chuẩn bị xong mọi thứ. Bố ngạc nhiên nhìn tôi, còn mẹ cười tủm tỉm. Tôi ríu rít: “Con muốn đến sớm nhất lớp.” Tôi háo hức tưởng tượng ra cảnh mình đến đầu tiên, cất tiếng chào thật to những bạn đến sau. Nhưng vừ

In [10]:
read_long_term_memory_by_session_id("session_20251028_105421")

2025-10-28T10:54:59.953644 session_20251028_105421 assistant : assistant: Xin chào! Tôi là trợ lý xưởng tóm tắt thông minh theo khối lớp. Tôi có thể giúp gì cho bạn?
2025-10-28T10:55:38.427037 session_20251028_105421 user : user: Tôi muốn tóm tắt văn bản tiếng Việt cho lớp 3
2025-10-28T10:55:38.856110 session_20251028_105421 user : user: Tôi muốn tóm tắt văn bản tiếng Việt cho lớp 3
2025-10-28T10:56:18.961364 session_20251028_105421 assistant : assistant: Rõ ràng! Bạn muốn tóm tắt văn bản tiếng Việt cho lớp 3, đúng không?

Vui mừng! Tôi sẽ giúp bạn thực hiện điều đó.

Văn bản của bạn có thể được đưa lên hình ảnh/PDF hay là text? (You can upload the file/image or type in the text)
2025-10-28T10:57:33.219459 session_20251028_105421 user : user: Văn bản của tôi như sau: Ngày khai trường đã đến. Sáng sớm, mẹ mới gọi một câu mà tôi đã vùng dậy, khác hẳn mọi ngày. Loáng một cái, tôi đã chuẩn bị xong mọi thứ. Bố ngạc nhiên nhìn tôi, còn mẹ cười tủm tỉm. Tôi ríu rít: “Con muốn đến sớm nhất lớp

In [17]:
coordinator_agent = create_react_agent(llm, [Coordinator_Agent.coordinator_tool], prompt) 
coordinator_agent_executor = AgentExecutor(agent=coordinator_agent, tools=[Coordinator_Agent.coordinator_tool], verbose=True, handle_parsing_errors=True) 

def call_coordinator_agent(state: AgentState):
    result = coordinator_agent_executor.invoke({"input": f"Hãy tạo bản tóm tắt cho bài viết sau: {state['input']}"})
    return {"messages": [f"Bản tóm tắt: {result['output']}"]}

abstracter_agent = create_react_agent(llm, [Abstracter_Agent.abstracter_tool], prompt) 
abstracter_agent_executor = AgentExecutor(agent=abstracter_agent, tools=[Abstracter_Agent.abstracter_tool], verbose=True, handle_parsing_errors=True) 

def call_abstracter_agent(state: AgentState):
    result = abstracter_agent_executor.invoke({"input": f"Hãy tạo bản tóm tắt trích xuất từ bài viết sau: {state['input']}"})
    return {"messages": [f"Bản tóm tắt trích xuất: {result['output']}"]}

extractor_agent = create_react_agent(llm, [Extractor_Agent.extractor_tool], prompt) 
extractor_agent_executor = AgentExecutor(agent=extractor_agent, tools=[Extractor_Agent.extractor_tool], verbose=True, handle_parsing_errors=True) 

def call_extractor_agent(state: AgentState):
    result = extractor_agent_executor.invoke({"input": f"Hãy tóm tắt diễn giải văn bản sau: {state['input']}"})
    return {"messages": [f"Tóm tắt diễn giải văn bản: {result['output']}"]}

grade_calibrator_agent = create_react_agent(llm, [GradeCalibrator_Agent.grade_calibrator_tool], prompt) 
grade_calibrator_agent_executor = AgentExecutor(agent=grade_calibrator_agent, tools=[GradeCalibrator_Agent.grade_calibrator_tool], verbose=True, handle_parsing_errors=True) 

def call_grade_calibrator_agent(state: AgentState):
    result = grade_calibrator_agent_executor.invoke({"input": f"Hãy điều chỉnh độ dài và từ vựng của bản tóm tắt sau: {state['input']}"})
    return {"messages": [f"Văn bản điều chỉnh: {result['output']}"]}

evaluator_agent = create_react_agent(llm, [Evaluator_Agent.evaluator_tool], prompt) 
evaluator_agent_executor = AgentExecutor(agent=evaluator_agent, tools=[Evaluator_Agent.evaluator_tool], verbose=True, handle_parsing_errors=True) 

def call_evaluator_agent(state: AgentState):
    result = evaluator_agent_executor.invoke({"input": f"Hãy đánh giá chất lượng và đưa ra điểm số dựa trên độ dễ hiểu của bản tóm tắt sau: {state['input']}"})
    return {"messages": [f"Đánh giá chất lượng: {result['output']}"]}

aggregator_agent = create_react_agent(llm, [Aggregator_Agent.aggregator_tool], prompt) 
aggregator_agent_executor = AgentExecutor(agent=aggregator_agent, tools=[Aggregator_Agent.aggregator_tool], verbose=True, handle_parsing_errors=True) 

def call_aggregator_agent(state: AgentState):
    result = aggregator_agent_executor.invoke({"input": f"Hãy tổng hợp tất cả các bản tóm tắt sau: {state['input']}"})
    return {"messages": [f"Tổng hợp tóm tắt: {result['output']}"]}

ocr_agent = create_react_agent(llm, [OCR_Agent.ocr_tool], prompt) 
ocr_agent_executor = AgentExecutor(agent=ocr_agent, tools=[OCR_Agent.ocr_tool], verbose=True, handle_parsing_errors=True) 

def call_ocr_agent(state: AgentState):
    result = ocr_agent_executor.invoke({"input": f"Hãy trích xuất văn bản từ hình ảnh, file, văn bản sau: {state['input']}"})
    return {"messages": [f"Văn bản trích xuất: {result['output']}"]}

spellchecker_agent = create_react_agent(llm, [SpellChecker_Agent.spellchecker_tool], prompt) 
spellchecker_agent_executor = AgentExecutor(agent=spellchecker_agent, tools=[SpellChecker_Agent.spellchecker_tool], verbose=True, handle_parsing_errors=True) 

def call_spellchecker_agent(state: AgentState):
    result = spellchecker_agent_executor.invoke({"input": f"Hãy kiểm tra và sửa lỗi chính tả của văn bản sau: {state['input']}"})
    return {"messages": [f"Văn bản sửa lỗi: {result['output']}"]}


AttributeError: module 'Source.ai.Multi_Agent.Source.Main.Agents.Agents_Summary.Coordinator_Agent' has no attribute 'coordinator_tool'

In [ ]:
graph_builder = StateGraph(AgentState)

graph_builder.add_node("coordinator_agent", Coordinator_Agent.coordinator_agent)
graph_builder.add_node("reader_ocr_agent", OCR_Agent.ocr_agent)
graph_builder.add_node("spellchecker_agent", SpellChecker_Agent.spellchecker_agent)
graph_builder.add_node("extractor_agent", Extractor_Agent.extractor_agent)
graph_builder.add_node("abstracter_agent", Abstracter_Agent.abstracter_agent)
graph_builder.add_node("grade_calibrator_agent", GradeCalibrator_Agent.grade_calibrator_agent)
graph_builder.add_node("evaluator_agent", Evaluator_Agent.evaluator_agent)
graph_builder.add_node("aggregator_agent", Aggregator_Agent.aggregator_agent)

graph_builder.set_entry_point("coordinator_agent")

graph_builder.add_edge("coordinator_agent", "reader_ocr_agent")
graph_builder.add_edge("reader_ocr_agent", "spellchecker_agent")
graph_builder.add_edge("spellchecker_agent", "extractor_agent")
graph_builder.add_edge("extractor_agent", "abstracter_agent")
graph_builder.add_edge("abstracter_agent", "grade_calibrator_agent")
graph_builder.add_edge("grade_calibrator_agent", "evaluator_agent")
graph_builder.add_edge("evaluator_agent", "aggregator_agent")
graph_builder.add_edge("aggregator_agent", END)

graph = graph_builder.compile()

input_test1 = "Tóm tắt bài viết sau: Ngày khai trường đã đến. Sáng sớm, mẹ mới gọi một câu mà tôi đã vùng dậy, khác hẳn mọi ngày. Loáng một cái, tôi đã chuẩn bị xong mọi thứ. Bố ngạc nhiên nhìn tôi, còn mẹ cười tủm tỉm. Tôi ríu rít: “Con muốn đến sớm nhất lớp.” Tôi háo hức tưởng tượng ra cảnh mình đến đầu tiên, cất tiếng chào thật to những bạn đến sau. Nhưng vừa đến cổng trường, tôi đã thấy mấy bạn cùng lớp đang ríu rít nói cười ở trong sân. Thì ra, không chỉ mình tôi muốn đến sớm nhất. Tôi chào mẹ, chạy ào vào cùng các bạn. Chúng tôi tranh nhau kể về chuyện ngày hè. Ngay cạnh chúng tôi, mấy em lớp 1 đang rụt rè níu chặt tay bố mẹ, thật giống tôi năm ngoái. Trước các em, tôi cảm thấy mình lớn lắm. Tôi đã là học sinh lớp 2 rồi cơ mà. "

final_state = graph.invoke({"input": input_test1})
print("Kết quả cuối cùng:")
for message in final_state["messages"]:
    print("- ", message)